In [ ]:
from google.colab import drive
drive.mount('/content/drive')

save_path = '/content/drive/MyDrive'

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install pymorphy2

     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 10.2 MB/s 


In [ ]:
from transformers import BertTokenizer
from transformers import AdamW, BertForMaskedLM
import torch
import numpy as np
import pymorphy2
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import pandas as pd
from keras.preprocessing.sequence import pad_sequences

random.seed(44)
morph = pymorphy2.MorphAnalyzer()
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
 
if device == torch.device('cpu'):
    print('Using cpu')
else:
    n_gpu = torch.cuda.device_count()
    print('Using {} GPUs'.format(torch.cuda.get_device_name(0)))

Using cpu


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForMaskedLM.from_pretrained("bert-base-multilingual-cased")

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
categories = ("животные", "музыка", "спорт", "литература")
categories_eng = ("animals", "music", "sport", "literature")

nouns_actors = []
for cat in categories_eng:
  f = open(save_path + f'/keywords/actors/true_keywords_nouns_actors_{cat}.txt')
  words = f.read()
  nouns_actors.append(words.split('\n')[:-1])

# В nouns не входят существительные, которые входят в actors (это я, по сути, пытался разбить слова на категории (вторая моя идея), но пока что только на две)
nouns = []
for ind, cat in enumerate(categories_eng):
  f = open(save_path + f'/keywords/nouns/true_keywords_nouns_{cat}.txt')
  words = f.read()
  words = words.split('\n')[:-1]
  nouns.append([])
  for word in words:
    if not word in nouns_actors[ind]:
      nouns[ind].append(word)

In [ ]:

# Недоделанное обучение берта, чтобы он замещал сразу много масок подряд
'''df = pd.read_csv(save_path + '/facts2.csv')

df'''

,texts,labels
0,1 Хамелеоны могут двигать глазами в разных нап...,0
1,2 Белка – лучший садовник. Миллионы деревьев в...,0
2,3 Зуб слона может весить до девяти килограмм!,0
3,"4 У млекопитающих кровь красная, у насекомых ж...",0
4,5 В среднем коровы какают 16 раз в день.,0
...,...,...
1176,Гимн Лиги чемпионов пришел в каждый дом благод...,2
1177,"Далеко не каждый знает, что самый популярный в...",2
1178,"Когда этот вид спорта только зарождался, то сп...",2
1179,"Первым спортсменом, позволившим себе нарушить ...",2


In [ ]:
'''alphabet = set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя ')

def generateTexts(texts, windowLen, only_long=True):
  out = []
  for text in texts:
    text = text.lower().replace('\n', ' ')
    text = ''.join(c for c in text if c in alphabet)
    text = text.split()
    if only_long: # Все примеры длины windowLen
      out.extend([' '.join(text[x:x+windowLen]) for x in range(0, len(text)-windowLen, windowLen)])
    else:
      out.extend([' '.join(text[x:x+windowLen]) for x in range(0, len(text), windowLen)])
  return out'''

In [ ]:
'''sentences = []
labels = []
 
for i in range(len(df.texts)):
  cat_texts = generateTexts([df.texts[i]], sentMaxLen, only_long=False)
  label = df.labels[i]
  for j in cat_texts:
    sentences.append(j)
    labels.append(label)
 
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
X_train = np.array(sentences)
Y_train = np.array(labels)'''

In [ ]:
'''train_texts = df['texts']

train_inputs = [tokenizer(sent, return_tensors="pt") for sent in df['texts']]
tokenized_texts = pad_sequences(
  tokenized_texts,
  maxlen=200,
  dtype=object,
  truncating="post",
  padding="post",
  value='[PAD]'
)
attention_masks = [[1 if i != '[PAD]' else 0 for i in seq] for seq in train_inputs]'''

In [ ]:
#sentence = '4.30. У слона пульс 20 ударов в минуту, а у паука на 40 ударов в минуту больше. Какой пульс у паука?'
#sentence = '	1.1. Летом в спортивный лагерь ходили 50 детей, из них 9 девочек. Сколько мальчиков ходили в спортивный лагерь?'
sentence = '	1.4. Во время летних соревнований по плаванию ребята посетили бассейн. Длина плавательной дорожки в бассейне 25 м. После того как первый участник соревнований проплыл часть дорожки, ему осталось проплыть 10 м. Сколько метров уже проплыл участник соревнований?'
category_from = 2
category_to = 0

In [ ]:
words_in_sentence = word_tokenize(sentence)
to_word = {}
count_masks = 0
for ind, word in enumerate(words_in_sentence):
  if morph.parse(word)[0].tag.POS == 'INFN' or morph.parse(word)[0].tag.POS == 'VERB':
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
    continue
  normal_word = morph.parse(word)[0].normal_form
  case_word = morph.parse(word)[0].tag.case
  number_word = morph.parse(word)[0].tag.number
  if normal_word in nouns_actors[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns_actors[category_to][random.randint(0, 19)]
    # Заменяю либо на существительное из другой категории, либо на маску (тестировал разные варианты)
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1
  if normal_word in nouns[category_from]:
    if to_word.get(normal_word) == None:
      to_word[normal_word] = nouns[category_to][random.randint(0, 19)]
    #words_in_sentence[ind] = morph.parse(to_word[normal_word])[0].inflect({number_word, case_word})[0]
    words_in_sentence[ind] = '[MASK]'
    count_masks += 1

In [ ]:
sentence = TreebankWordDetokenizer().detokenize(words_in_sentence)

sentence

'1.4 . Во время летних [MASK] по [MASK] [MASK] [MASK] [MASK]. Длина плавательной [MASK] в [MASK] 25 м. После того как первый [MASK] [MASK] [MASK] часть [MASK], ему [MASK] [MASK] 10 м. Сколько метров уже [MASK] [MASK] [MASK]?'

In [ ]:
for cnt in range(count_masks):
  tokenized = tokenizer(sentence, return_tensors="pt")
  with torch.no_grad():
    outputs = model(**tokenized)
  masked_index = np.where(torch.flatten(tokenized['input_ids']).numpy() == 103)
  for i in masked_index[0]:
    predicted_index = torch.argmax(outputs[0][0][i]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    ind = sentence.split().index('[MASK]')
    sentence = sentence.split()
    sentence[ind] = predicted_token
    sentence = ' '.join(sentence)

ValueError: ignored

In [ ]:
sentence

'1.4 . Во время летних Олимпийских по ##им . М [MASK]. Длина плавательной . в . 25 м. После того как первый . . . часть [MASK], ему . . 10 м. Сколько метров уже . . [MASK]?'